This notebook is based on [this example](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb#scrollTo=Fqo_hMRZiubM)

In [2]:
!pip install faiss-cpu langchain langchain-community langgraph langchain-openai langchain-anthropic -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50

In [4]:
import os
from dotenv import load_dotenv
from typing import Optional

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from langchain_core.runnables import RunnablePassthrough # For LCEL
from google.colab import userdata

# Load environment variables (for API key)
load_dotenv()

# 1. Define the desired output structure using Pydantic
class EvaluationResult(BaseModel):
    score: int = Field(description="The score from 1 to 5.")
    reasoning: str = Field(description="A brief explanation for the assigned score.")

    @validator('score')
    def score_must_be_in_range(cls, v):
        if not 1 <= v <= 5:
            raise ValueError('Score must be between 1 and 5')
        return v

# 2. Set up the PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=EvaluationResult)

# 3. Create the PromptTemplate
prompt_template_str = """
You are an impartial AI judge in TAX. Your task is to evaluate the quality of a submitted answer compared to a golden (correct) answer for a given question.
Please provide a score from 1 to 5 based on the following criteria:

1:  Completely Incorrect - The submitted answer is entirely wrong, irrelevant, or fails to address the question.
2:  Mostly Incorrect - The submitted answer has significant errors or omissions, though it might show a slight understanding or attempt.
3:  Partially Correct - The submitted answer is partially correct but contains notable inaccuracies, misses key points, or lacks sufficient detail. It addresses the main aspects of the question but has clear room for improvement.
4:  Mostly Correct - The submitted answer is largely correct and addresses the question well, with only minor inaccuracies, omissions, or stylistic issues.
5:  Fully Correct - The submitted answer is accurate, complete, and aligns perfectly or very closely with the golden answer. It demonstrates a thorough understanding.

You MUST provide your output in the specified JSON format. Do not add any other text before or after the JSON.

Question:
{question}

Golden Answer:
{golden_answer}

Submitted Answer to Evaluate:
{submitted_answer}

{format_instructions}
"""

prompt = PromptTemplate(
    template=prompt_template_str,
    input_variables=["question", "golden_answer", "submitted_answer"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 4. Initialize the LLM
# You can choose different models like "gpt-4", "gpt-3.5-turbo-0125", etc.
# temperature=0 makes the output more deterministic and suitable for evaluation
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-mini-2025-04-14",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
# For potentially better (but more expensive) judging, consider "gpt-4-turbo-preview" or "gpt-4"
# llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0)

# 5. Create the evaluation chain using LCEL
# The chain will take a dictionary of inputs, format them into the prompt,
# pass it to the LLM, and then parse the LLM's output.
evaluation_chain = prompt | llm | parser

# --- Example Usage ---
def evaluate_answer(question: str, golden_answer: str, submitted_answer: str) -> Optional[EvaluationResult]:
    """
    Evaluates a submitted answer against a golden answer using an LLM judge.
    """
    if not os.getenv("OPENAI_API_KEY"):
        print("Error: OPENAI_API_KEY not found. Please set it in your .env file or environment.")
        return None

    try:
        print(f"\n--- Evaluating ---")
        print(f"Question: {question}")
        print(f"Golden Answer: {golden_answer}")
        print(f"Submitted Answer: {submitted_answer}")

        result = evaluation_chain.invoke({
            "question": question,
            "golden_answer": golden_answer,
            "submitted_answer": submitted_answer
        })
        return result
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")
        # Potentially try to parse the raw output if Pydantic fails
        # For example, if the LLM didn't perfectly follow JSON format.
        # This is more advanced error handling.
        print(f"Raw LLM output might be available in the exception details or by running llm.invoke directly.")
        return None

if __name__ == "__main__":
    # Example 1: Good answer
    q1 = "What is the capital of France?"
    ga1 = "The capital of France is Paris."
    sa1_good = "Paris is the capital of France."
    result1 = evaluate_answer(q1, ga1, sa1_good)
    if result1:
        print(f"Score: {result1.score}, Reasoning: {result1.reasoning}")



<ipython-input-4-5bf2d8a451a7>:20: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('score')



--- Evaluating ---
Question: What is the capital of France?
Golden Answer: The capital of France is Paris.
Submitted Answer: Paris is the capital of France.
Score: 5, Reasoning: The submitted answer correctly identifies Paris as the capital of France, matching the golden answer in meaning and completeness, just phrased differently.


In [5]:
!git clone https://github.com/RadchaneepornC/OfficeBuddyPrime_LLMAgentic

Cloning into 'OfficeBuddyPrime_LLMAgentic'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (264/264), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 264 (delta 106), reused 192 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (264/264), 25.85 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [17]:
import json
import os


def read_json_file(file_path):

  try:
    with open(file_path, 'r') as f:
      sample_records = json.load(f)
      return sample_records
  except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'. Please check the file path.")
  except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in '{file_path}'. Please check the file content.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")





In [18]:
!ls -r OfficeBuddyPrime_LLMAgentic/methods/rag

rag_test_ong_version.ipynb  evaluation_results.json
rag-test_bay_version.ipynb  evaluation


In [22]:
test_sample = read_json_file("OfficeBuddyPrime_LLMAgentic/methods/rag/evaluation_results.json")

In [30]:
eval = [ ]
for sample in test_sample['test_cases']:
  result, golden_answer = sample['inference_result'], sample['gold_answer']
  question = sample['question']
  score = evaluate_answer(question, golden_answer, result)
  eval.append((score.score, score.reasoning))



--- Evaluating ---
Question: ใครบ้างที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?
Golden Answer: ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล
Submitted Answer: จากข้อมูลที่ค้นหาได้ ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้:

1. บุคคลธรรมดา
2. ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล
3. ผู้ถึงแก่ความตายระหว่างปีภาษี
4. กองมรดกที่ยังไม่ได้แบ่ง
5. วิสาหกิจชุมชน ตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญ หรือคณะบุคคลที่มิใช่นิติบุคคล

เงินได้ที่ต้องเสียภาษีเรียกว่า "เงินได้พึงประเมิน" ซึ่งหมายถึงเงินได้ที่เกิดขึ้นระหว่างวันที่ 1 มกราคม ถึง 31 ธันวาคม ของปีใดๆ ประกอบด้วย:


In [32]:
sum([ score for score, _ in eval ])/len(eval)

4.8

In [35]:
!ls OfficeBuddyPrime_LLMAgentic/methods/vanilla_inference_with_tools

evaluation_result    inference.py  tax-test-results.json
inference_gradio.py  raw_data


In [36]:
test_sample = read_json_file("OfficeBuddyPrime_LLMAgentic/methods/vanilla_inference_with_tools/tax-test-results.json")

In [37]:
eval = [ ]
for sample in test_sample['test_cases']:
  result, golden_answer = sample['inference_result'], sample['golden_answer']
  question = sample['question']
  score = evaluate_answer(question, golden_answer, result)
  eval.append((score.score, score.reasoning))



--- Evaluating ---
Question: ใครบ้างที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?
Golden Answer: ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล
Submitted Answer: # ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา

ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้

1. บุคคลธรรมดา
2. ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล  
3. ผู้ถึงแก่ความตายระหว่างปีภาษี
4. กองมรดกที่ยังไม่ได้แบ่ง
5. วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล

โดยหากมีเงินได้พึงประเมินที่ได้รับในระหว่างปีภาษี (1 มกราคม - 31 ธันวาคม) ถึงเกณฑ์ที่กฎหมายกำหนด ก็จะต้องยื่นแบบแสดงรา

In [38]:
sum([ score for score, _ in eval ])/len(eval)

4.5

In [40]:
test_sample = read_json_file("OfficeBuddyPrime_LLMAgentic/methods/naive/evaluation_results_naive.json")

In [41]:
eval = [ ]
for sample in test_sample['test_cases']:
  result, golden_answer = sample['inference_result'], sample['gold_answer']
  question = sample['question']
  score = evaluate_answer(question, golden_answer, result)
  eval.append((score.score, score.reasoning))



--- Evaluating ---
Question: ใครบ้างที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?
Golden Answer: ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล
Submitted Answer: บุคคลที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่:

1. บุคคลธรรมดาทั่วไป (บุคคลที่มีเงินได้)
2. ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่ไม่ใช่นิติบุคคล
3. ผู้ถึงแก่ความตายระหว่างปีภาษี
4. กองมรดกที่ยังไม่ได้แบ่ง

โดยทั่วไป บุคคลที่มีเงินได้พึงประเมินในปีภาษีเกินกว่าเกณฑ์ที่กำหนด (ขึ้นอยู่กับค่าใช้จ่ายและค่าลดหย่อนต่างๆ) มีหน้าที่ต้องยื่นแบบแสดงรายการและเสียภาษีเงินได้บุคคลธรรมดา ทั้งนี้ เงินได้พึงประเมินหมายถึงเงินได้ประเภทต่างๆ ตามที่กฎหมายกำหนด เช่น เงินเดือน ค่าจ้าง เงินโบนัส รายได้จากวิชาชีพอิสระ รายได้จากธุรกิจ ดอกเบี้ย เงิ

In [42]:
sum([ score for score, _ in eval ])/len(eval)

3.7